In [18]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [19]:
pickle_file = '/Users/shivankurkapoor/Desktop/Study/ML/Udacity/Assignment1/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [20]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    #Flattening the image matrix
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [22]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    #Constants
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset, :])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    #Weight matrix is initialized with truncated Normaal Distribution
    #Baises are initialized to zero
    weights = tf.Variable(
        tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Training
    #The logits are passed through softmax function and then we apply
    #the cross-entropy loss and optimize it
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #Optimizer - Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)
    
    

In [27]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
           / predictions.shape[0])


with tf.Session(graph = graph) as session:
    #Initialization
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_predictions])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_predictions.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
            test_predictions.eval(), test_labels))        

Initialized
Loss at step 0: 17.255079
Training accuracy: 7.1%
Validation accuracy: 10.0%
Loss at step 100: 2.289880
Training accuracy: 71.8%
Validation accuracy: 70.9%
Loss at step 200: 1.867652
Training accuracy: 74.8%
Validation accuracy: 73.1%
Loss at step 300: 1.636061
Training accuracy: 76.3%
Validation accuracy: 73.9%
Loss at step 400: 1.478990
Training accuracy: 77.2%
Validation accuracy: 74.3%
Loss at step 500: 1.360870
Training accuracy: 78.1%
Validation accuracy: 74.6%
Loss at step 600: 1.266835
Training accuracy: 78.6%
Validation accuracy: 74.7%
Loss at step 700: 1.189453
Training accuracy: 79.0%
Validation accuracy: 75.0%
Loss at step 800: 1.124198
Training accuracy: 79.4%
Validation accuracy: 75.2%
Test accuracy: 82.8%


In [28]:
'''Using Stochastic Gradient Descent'''
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

In [29]:
num_steps = 3001
with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #Generating a mini match
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" %(step, l))
            print("Minimatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(
                test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.402122
Minimatch accuracy: 13.3%
Validation accuracy: 16.2%
Test accuracy: 17.5%
Minibatch loss at step 500: 1.270015
Minimatch accuracy: 81.2%
Validation accuracy: 75.8%
Test accuracy: 83.0%
Minibatch loss at step 1000: 1.394333
Minimatch accuracy: 79.7%
Validation accuracy: 76.1%
Test accuracy: 84.0%
Minibatch loss at step 1500: 0.704445
Minimatch accuracy: 82.8%
Validation accuracy: 76.6%
Test accuracy: 84.3%
Minibatch loss at step 2000: 0.935610
Minimatch accuracy: 81.2%
Validation accuracy: 77.3%
Test accuracy: 85.0%
Minibatch loss at step 2500: 0.794369
Minimatch accuracy: 77.3%
Validation accuracy: 77.9%
Test accuracy: 85.7%
Minibatch loss at step 3000: 0.912315
Minimatch accuracy: 78.1%
Validation accuracy: 78.6%
Test accuracy: 86.1%


In [33]:
#TRAINING A SINGLE LAYER NN WITH 1024 UNITS

In [40]:
num_nodes = 1024
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    #Placeholders
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    #Training 
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, 
                                                                  labels=tf_train_labels))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for training
    train_prediction = tf.nn.softmax(logits_2)
    
    #Preditions for validation
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    valid_prediction = tf.nn.softmax(logits_2)
    
    #Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    test_prediction = tf.nn.softmax(logits_2)
    

In [43]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #Generating a mini match
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" %(step, l))
            print("Minimatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(
                test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 347.928772
Minimatch accuracy: 11.7%
Validation accuracy: 22.8%
Test accuracy: 24.0%
Minibatch loss at step 500: 21.824863
Minimatch accuracy: 78.9%
Validation accuracy: 79.7%
Test accuracy: 87.2%
Minibatch loss at step 1000: 10.930933
Minimatch accuracy: 78.9%
Validation accuracy: 81.2%
Test accuracy: 88.5%
Minibatch loss at step 1500: 6.185275
Minimatch accuracy: 85.9%
Validation accuracy: 79.7%
Test accuracy: 86.4%
Minibatch loss at step 2000: 3.352630
Minimatch accuracy: 89.1%
Validation accuracy: 82.1%
Test accuracy: 89.2%
Minibatch loss at step 2500: 2.853768
Minimatch accuracy: 87.5%
Validation accuracy: 81.5%
Test accuracy: 89.2%
Minibatch loss at step 3000: 2.094995
Minimatch accuracy: 82.8%
Validation accuracy: 82.1%
Test accuracy: 89.6%
